In [19]:
import os
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings;warnings.simplefilter('ignore')
root = '../data/phase_2/raw/other/'
os.listdir(root)

['200mb201701r2.csv',
 '200mb201909.csv',
 'Aroma-1 Unit 000 Split_Factor Calculation.xlsx',
 'dcs data.xlsx',
 'dcs datar1.xlsx',
 'FCFC Import Naphtha Composition Lab_001~100_R2.xlsx',
 'lims data.xlsx',
 '~$lims data.xlsx',
 '說明.txt',
 '附件一.xlsx',
 '附件二.xlsx',
 '預餾案dashboard草圖.pdf']

In [20]:
df = pd.read_excel(root+'附件一.xlsx')
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,ProII Composition,Input,NaN,Composition for Reformer Model,Output
1,NaN,Oxygen,0,NaN,Hydrogen,0
2,NaN,Methane,0,NaN,Methane,0
3,NaN,Ethane,0,NaN,Ethane,0
4,NaN,Propane,2.04389e-05,NaN,Propane,2.04389e-05
5,NaN,n-Butane,0.000136193,NaN,n-Butane,5.44771e-05
6,NaN,i-Pentane,0.00187765,NaN,i-Butane,7.4906e-05
7,NaN,n-Pentane,0.00531037,NaN,C4 Naphthenes,6.80964e-06
8,NaN,tr2-Pentene,0.000100135,NaN,n-Pentane,0.00531037
9,NaN,Cyclopentane,0.00192004,NaN,C5 i-Paraffins,0.00197778


In [21]:
x_col = [ i.strip() for i in df.iloc[1:-1,1].values]
x_col[0],x_col[-1]

('Oxygen', 'n-Tridecane')

In [22]:
X = df.iloc[1:-1,2].values

In [23]:
y_col = [ i.strip() for i in df.iloc[1:1+33,4].values]
y_col[0],y_col[-1]

('Hydrogen', 'C10+ Aromtics')

In [24]:
Y = df.iloc[1:1+33,5].values

In [25]:
W = pd.DataFrame(index=x_col,columns=y_col).fillna(0)
W.index.tolist().index('Ethylcyclohexane')

28

In [8]:
def fill_W_value(W):
    W.loc['Methane','Methane'] = 1
    W.loc['Ethane','Ethane'] = 1
    W.loc['Propane','Propane'] = 1
    W.loc['n-Butane',['n-Butane','i-Butane','C4 Naphthenes']] = [0.4,0.55,0.05]
    W.loc['n-Pentane','n-Pentane'] = 1
    
    W.loc[['i-Pentane','tr2-Pentene'],'C5 i-Paraffins'] = [1,1]
    W.loc['Cyclopentane','C5 Naphthenes'] = 1
    W.loc['n-Hexane','n-Hexane'] = 1
    W.loc[['3-Methylpentane','tr2-Hexene'],'C6 i-Paraffins'] = [1,1]
    W.loc[['Methylcyclopentane','Cyclohexane'],'C6 Naphthenes'] = [1,1]
    
    W.loc['Benzene','Benzene'] = 1
    W.loc['n-Heptane','n-Heptane'] = 1
    W.loc[['2,2-Dimethylpentane','2,2,3-Trimethylbutane','4-Methyl-1-hexene','2-Methylhexane'],'C7 i-Paraffins'] = [1,1,1,1]
    W.loc[['t-1,3-Dimethylcyclopentane','Methylcyclohexane','Ethylcyclopentane'],'C7 Naphthenes'] = [1,1,1]
    W.loc['Toluene','Toluene'] = 1
    
    W.loc['n-Octane','n-Octane'] = 1
    W.loc[['2,3-Dimethyl-1-hexene','2-Methylheptane'],'C8 i-Paraffins'] = [1,1]
    W.loc['Ethylcyclohexane','C8 Naphthenes'] = 1
    W.loc['EthylBenzene','Ethylbenzene'] = 1
    W.loc['P-Xylene','P-Xylene'] = 1
    
    W.loc['M-Xylene','M-Xylene'] = 1
    W.loc['O-Xylene','O-Xylene'] = 1
    W.loc['n-Nonane','n-Nonane'] = 1
    W.loc[['2-Methyloctane','1-Nonene'],'C9 i-Paraffins'] = [1,1]
    W.loc[['1-trans-3,5-Trimethylcyclohexane','n-Propylcyclohexane'],'C9 Naphthenes'] = [1,1]
    
    W.loc[['i-Propylbenzene','n-Propylbenzene','1,3,5-Trimethylbenzene','1-Methyl-2-ethylbenzene','1,2,3-Trimethylbenzene'],'C9 Aromtics'] = [1,1,1,1,1]
    W.loc[['n-Decane','n-Undecane','n-Dodecane','n-Tridecane'],'C10+ n-Paraffins'] = [1,1,1,1]
    W.loc[['2,4-Dimethyloctane','2,2,3,3-Tetramethylhexane','3-Methylnonane'],'C10+ i-Paraffins'] = [1,1,1]
    W.loc['tert-Butylcyclohexane','C10+ Naphthenes'] = 1
    W.loc[['1,2-Diethylbenzene','n-Pentylbenzene','n-Hexylbenzene'],'C10+ Aromtics'] = [1,1,1]
    
    return W.values

In [9]:
W = fill_W_value(W)

# CHECK

In [10]:
assert (X@W == Y).sum() == 33
X@W == Y

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [11]:
class transformer_5433(object):
    def __init__(self,x_col,y_col,W):
        self.x_col = x_col
        self.y_col = y_col
        self.W = W
    
    def __call__(self,X):
        return pd.DataFrame(X.values @ self.W,columns=self.y_col)

In [12]:
tr = transformer_5433(x_col,y_col,W)

In [13]:
X = pd.DataFrame(X.reshape(1,-1),columns=x_col)
X

,Oxygen,Methane,Ethane,Propane,n-Butane,i-Pentane,n-Pentane,tr2-Pentene,Cyclopentane,3-Methylpentane,...,"2,2,3,3-Tetramethylhexane",3-Methylnonane,tert-Butylcyclohexane,n-Decane,"1,2-Diethylbenzene",n-Undecane,n-Pentylbenzene,n-Dodecane,n-Hexylbenzene,n-Tridecane
0,0,0,0,2.04389e-05,0.000136193,0.00187765,0.00531037,0.000100135,0.00192004,0.286566,...,0.946327,2.1055,0.352242,0.498155,0.241376,0.0884591,0.0135893,0.0012702,0.00252418,0


In [14]:
Y = pd.DataFrame(Y.reshape(1,-1),columns=y_col)
Y

,Hydrogen,Methane,Ethane,Propane,n-Butane,i-Butane,C4 Naphthenes,n-Pentane,C5 i-Paraffins,C5 Naphthenes,...,M-Xylene,O-Xylene,n-Nonane,C9 i-Paraffins,C9 Naphthenes,C9 Aromtics,C10+ n-Paraffins,C10+ i-Paraffins,C10+ Naphthenes,C10+ Aromtics
0,0,0,0,2.04389e-05,5.44771e-05,7.4906e-05,6.80964e-06,0.00531037,0.00197778,0.00192004,...,2.06911,1.71304,5.36032,9.9227,4.58693,2.57461,0.587885,3.71281,0.352242,0.257489


In [15]:
y_pred = tr(X)
y_pred

,Hydrogen,Methane,Ethane,Propane,n-Butane,i-Butane,C4 Naphthenes,n-Pentane,C5 i-Paraffins,C5 Naphthenes,...,M-Xylene,O-Xylene,n-Nonane,C9 i-Paraffins,C9 Naphthenes,C9 Aromtics,C10+ n-Paraffins,C10+ i-Paraffins,C10+ Naphthenes,C10+ Aromtics
0,0,0,0,2.04389e-05,5.44771e-05,7.4906e-05,6.80964e-06,0.00531037,0.00197778,0.00192004,...,2.06911,1.71304,5.36032,9.9227,4.58693,2.57461,0.587885,3.71281,0.352242,0.257489


In [16]:
assert (y_pred == Y).sum().sum() == 33

In [17]:
joblib.dump(tr,'../model/transformer(54_to_33).pkl')
print('save done!')

save done!


In [18]:
os.listdir('../model')

['ANN(58_to_sp162(二合一)).pkl',
 'ANN(58_to_sp162).pkl',
 'ANN(58_to_y162(二合一)).pkl',
 'ANN(58_to_y162).pkl',
 'ANN(energy).pkl',
 'ANN(tray).pkl',
 'eva.pkl',
 'OperationOptimModel.pkl',
 'SVR(4_to_23).pkl',
 'transformer(23_to_54).pkl',
 'transformer(33_to_15).pkl',
 'transformer(43_to_33).pkl',
 'transformer(54_to_23).pkl',
 'transformer(54_to_33).pkl',
 'transformer(SP162_to_Y162).pkl']